In [115]:
import os

# Create a folder inside /content
os.makedirs("/content/my_folder", exist_ok=True)

print("Folder created at:", os.path.abspath("/content/my_folder"))


Folder created at: /content/my_folder


Task 1: conversation management & summarization

In [116]:
import time
import re

In [117]:
chat_history = []
def add_message(role, text):
    chat_history.append({
        "role": role,
        "text": text,
        "time": time.strftime("%Y-%m-%d %H:%M:%S")
    })

In [118]:
stopwords = {"the","is","and","a","an","of","in","on","to","for","with","at","by","it"}
def tokenize(text):
    text = text.lower()
    # print("Lowercased:", text)

    text = re.sub(r"[^a-z0-9\s]", "", text) #removing puntuations
    # print("Cleaned:", text)

    words = re.findall(r"\b\w+\b", text)
    tokens = [w for w in words if w not in stopwords]

    return tokens

In [119]:
# step 1: split into sentences
# step 2: count word frequency
# step 3: give each sentence a score
# step 4: pick top sentences

def split_sentences(text):
    return re.split(r'[.!?]', text)

def summarize_history(max_sent=2):
    #step-1
    all_text = " ".join([m["text"] for m in chat_history if m["role"]!="system"])
    sentences = split_sentences(all_text)

    #step-2
    freq = {}
    for w in tokenize(all_text):
        freq[w] = freq.get(w,0)+1

    #step-3
    scores = {}
    for s in sentences:
        score = 0
        for w in tokenize(s):
            score =score+ freq.get(w,0)
        scores[s] = score

    #step-4
    sorted_sents = sorted(scores.items(), key=lambda x:x[1], reverse=True)
    top_sents = [s for s,_ in sorted_sents[:max_sent]]
    return " ".join(top_sents).strip()


In [120]:
run_count = 0
k_value = 3   # after every 3 runs, summarize

def add_turn(user_msg, assistant_msg):
    global run_count
    add_message("user", user_msg)
    add_message("assistant", assistant_msg)
    run_count += 1

    if run_count % k_value == 0:
        summary = summarize_history(max_sent=2)
        chat_history.clear()
        add_message("system", "Summary: " + summary)


In [121]:
def truncate_by_turns(n):
    return chat_history[-n:]

def truncate_by_words(max_words):
    out = []
    total = 0
    for m in reversed(chat_history):
        words = m["text"].split()
        if total + len(words) <= max_words:
            out.insert(0, m)
            total += len(words)
        else:
            break
    return out

def truncate_by_chars(max_chars):
    out = []
    total = 0
    for m in reversed(chat_history):
        if total + len(m["text"]) <= max_chars:
            out.insert(0, m)
            total += len(m["text"])
        else:
            break
    return out


In [122]:
#display fucntion
def show_chat():
    for i, msg in enumerate(chat_history, 1):
        print(f"{i}. [{msg['role']}] {msg['text']} ({msg['time']})")


In [123]:
# now i will test the functions step by step

add_turn("Hi, I am Ajay. Laptop battery draining fast.", "Oh sorry, since when?")
add_turn("Around 1 year. It lasts 1 hour only.", "Ok, does charging light stay solid?")
add_turn("Yes, but laptop heats up too.", "Sounds like battery issue, may need replacement.")

print("Chat after 3rd run (should be summarized):")
show_chat()

# add more to see history grow again
add_turn("Can I claim warranty?", "Depends, when did you buy?")
add_turn("Bought online on Aug 2024. My email is ajaygoud738780@gmail.com", "Ok, let's escalate to team.")

print("\nNow full chat history:")
show_chat()

print("\nTruncate by last 4 messages:")
for msg in truncate_by_turns(4):
    print(msg)

print("\nTruncate by 30 words:")
for msg in truncate_by_words(30):
    print(msg)

print("\nTruncate by 150 characters:")
for msg in truncate_by_chars(150):
    print(msg)


Chat after 3rd run (should be summarized):
1. [system] Summary: Sounds like battery issue, may need replacement  Yes, but laptop heats up too (2025-09-13 18:33:57)

Now full chat history:
1. [system] Summary: Sounds like battery issue, may need replacement  Yes, but laptop heats up too (2025-09-13 18:33:57)
2. [user] Can I claim warranty? (2025-09-13 18:33:57)
3. [assistant] Depends, when did you buy? (2025-09-13 18:33:57)
4. [user] Bought online on Aug 2024. My email is ajaygoud738780@gmail.com (2025-09-13 18:33:57)
5. [assistant] Ok, let's escalate to team. (2025-09-13 18:33:57)

Truncate by last 4 messages:
{'role': 'user', 'text': 'Can I claim warranty?', 'time': '2025-09-13 18:33:57'}
{'role': 'assistant', 'text': 'Depends, when did you buy?', 'time': '2025-09-13 18:33:57'}
{'role': 'user', 'text': 'Bought online on Aug 2024. My email is ajaygoud738780@gmail.com', 'time': '2025-09-13 18:33:57'}
{'role': 'assistant', 'text': "Ok, let's escalate to team.", 'time': '2025-09-13 18:33:

Task 2: JSON Schema Classification & Information **Extraction**

In [ ]:
import json
from getpass import getpass
from openai import OpenAI

key = getpass("Enter your API key: ")
client = OpenAI(api_key=key, base_url="https://api.groq.com/openai/v1")

In [ ]:
functions = [
    {
        "name": "extract_user_info",
        "description": "Extract basic user info from conversation",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "phone": {"type": "string"},
                "location": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": ["name","email","phone","location","age"]
        }
    }
]


In [ ]:
def extract_info_from_chat(chat_text):
    try:
        response = client.chat.completions.create(
            model="openai/gpt-oss-20b",
            messages=[{"role":"user","content":chat_text}],
            functions=functions,
            function_call={"name":"extract_user_info"}
        )
        args = response.choices[0].message.function_call.arguments
        return json.loads(args)
    except Exception as e:
        print("error:", e)
        return None


In [ ]:
from logging import exception
# sample chats

chat1 = "Hello, my name is Ajay. I am 21 years old. I live in Bangalore. My email is ajay738780@gmail.com and my phone is 6305738780."
chat2 = "Hey, I am Priya from Delhi, 19 years old. You can reach me at priya99@yahoo.com or call 0123456789."
chat3 = "This is John. Age 25. Based in Mumbai. Email: john_doe@hotmail.com. Phone: 9998887776."

for i, chat in enumerate([chat1, chat2, chat3], 1):
    print(f"--- Chat {i} ---")
    extracted = extract_info_from_chat(chat)
    print("Extracted:", extracted)

    if extracted:
        valid = all(k in extracted and extracted[k] for k in ["name","email","phone","location","age"])
        print("Valid?", valid)
    else:
        print("Extraction failed, skipping validation.")
    print()
